In [ ]:
# Try to append data to existing excel file, while checking first if file there try find file, 
# if found append excel method else create file
# Create info essentials, test new info def
# Check energy settings

In [1]:
from random import randint
from time import sleep
import datetime
import yfinance as yf
import openpyxl
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Def for reading the ticker file as csv: csv_ticker('filename.csv', 'columnname')
def csv_ticker(csv_file, ticker_col):
    df = pd.read_csv(csv_file)
    df_tick = df[ticker_col].drop_duplicates(keep='first').dropna()
    return df_tick

# Def for reading the ticker file as xlsx: xlsx_ticker('filename.xlsx', 'columnname') returns "df_tick"
def xlsx_ticker(xlsx_file, ticker_col):
    df = pd.read_excel(xlsx_file)
    df_tick = df[ticker_col].drop_duplicates(keep='first').dropna()
    return df_tick

# Def for reading collected data after preprocessing through “def info()“ and “def fin()“
def csv_df(csv_file):
    df = pd.read_csv(csv_file)
    return df

def xlsx_df(xlsx_file):
    df = pd.read_excel(xlsx_file)
    return df

In [3]:
# Analysis constists of 3 tables: stock advisor company info = df_1, 
# SnP500 company info = df_2, stock tips = df_adv
df_1 = xlsx_df('test.info.xlsx')
df_2 = csv_df('df_info_465_2021.10.04 21_31_53.csv')
df_adv = csv_df('advisor stocks.csv')

# Company info tables are appended
df_1 = df_1.append(df_2)

# Duplicates are dropped to join with stock tips df_adv
df_1 = df_1.drop_duplicates(subset=['symbol'])
df_1.shape


# Stock tips df_adv has to be prepared for left join, renaming, drop duplicates

# rename key_column
df_adv = df_adv.rename(columns={'ticker': 'symbol', 'symbol':'symbol_'})

# drop duplicates with same symbol/ ticker
df_adv = df_adv.drop_duplicates(subset=['symbol'])
df_adv

# left join on 'symbol' getting columns 'status_advisor', 'date_advisor', 'price_when_advised', 'source_advisor'
df_1 = pd.merge(df_1, df_adv[['symbol','status_advisor','date_advisor',
                             'price_when_advised','source_advisor']],on='symbol', how='left')

In [ ]:
# Storing the result to prepare df for K-means
now = datetime.datetime.now()
now = now.strftime("%Y.%m.%d %H_%M_%S")
len_df_1 = str(len(df_1))
#df_1.to_excel('df_1'+len_df+'_'+now+'.xlsx')

In [4]:
# Selecting columns for K-means clustering ('priceToBook', 'trailingAnnualDividendRate','debtToEquity',)
df_k = df_1[['sector','city','state','country','industry','financialCurrency','shortName','longName','symbol',
             'revenueGrowth', 
             'payoutRatio',  'trailingPE', 'marketCap',]]

# Filling the dividend information with 0 instead of nan | Watch out warnings override if there is an error
# df_k[['trailingAnnualDividendRate', 'payoutRatio']] = df_k[['trailingAnnualDividendRate', 'payoutRatio']].fillna(0)
df_k['payoutRatio'] = df_k['payoutRatio'].fillna(0)

# Dropping the other nan values for now
df_k = df_k.dropna()

# Taking the numerical columns --> select_dtypes(include=[np.number]).dtypes).
# Storing the numerical df as X to move on with standardization, as the data is wide spread
df_k_num = df_k.select_dtypes(include=['float64'])
X = df_k_num

/var/folders/5q/m0yt4w2n3279165l93qszgp00000gn/T/ipykernel_45036/3246516118.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_k['payoutRatio'] = df_k['payoutRatio'].fillna(0)


In [5]:
X.shape

(662, 4)

In [6]:
sns.pairplot(X)
plt.show()

KeyboardInterrupt: 

In [ ]:
# 1. Scaling features

scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns = X.columns)
display(X.head())
print()
display(X_scaled_df.head())
X_scaled_df.describe()

In [ ]:
# Defining the KMeans https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
kmeans = KMeans(n_clusters=8, init="k-means++", n_init=100, max_iter=300, tol=0, algorithm="elkan", 
                random_state=1234)
kmeans.fit(X_scaled_df)

print(len(kmeans.labels_))

# assign a cluster to each example
labels = kmeans.predict(X_scaled_df)

# retrieve unique clusters
clusters = np.unique(labels)

# create scatter plot for samples from each cluster
for cluster in clusters:
    # get row indexes for samples with this cluster
    row_ix = np.where(labels == cluster)
    # create scatter of these samples
    plt.scatter(X.to_numpy()[row_ix, 1], X.to_numpy()[row_ix, 2])

# show the plot
plt.show()

In [ ]:
# Review clusters and observations per cluster
clusters = kmeans.predict(X_scaled_df)
#clusters
pd.Series(clusters).value_counts().sort_index()

In [ ]:
# Assigning back the clusters to the companies

#X_df = pd.DataFrame(X)
df_k["cluster"] = clusters
df_k.head()

In [ ]:
# Parameter tuning - first checking the inertia, then checking different settings
print('inertia of kmeans1:',kmeans.inertia_)

kmeans2 = KMeans(n_clusters=8,
                init="k-means++",
                n_init=100,  # try with 1, 4, 8, 20, 30, 100...
                max_iter=20,
                tol=0,
                algorithm="elkan",
                random_state=1234)
kmeans2.fit(X_scaled_df)
print('inertia of kmeans2:',kmeans2.inertia_)

In [ ]:
# Finding k with the elbow method

K = range(2, 20)
inertia = []

for k in K:
    kmeans = KMeans(n_clusters=k,
                    random_state=1234)
    kmeans.fit(X_scaled_df)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(18,10))
plt.plot(K, inertia, 'bx-')
plt.xlabel('k')
plt.ylabel('inertia')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Elbow Method displaying the optimal k')

In [ ]:
# Finding k with the Silhouette Method

K = range(2, 20)
silhouette = []

for k in K:
    kmeans = KMeans(n_clusters=k,
                    random_state=1234)
    kmeans.fit(X_scaled_df)
    silhouette.append(silhouette_score(X_scaled_df, kmeans.predict(X_scaled_df)))

plt.figure(figsize=(18,10))
plt.plot(K, silhouette, 'bx-')
plt.xlabel('k')
plt.ylabel('silhouette score')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Silhouette Method displaying the optimal k')

In [ ]:
# TESTING with samples
# splitting dataframe by row index to create test samples for data downloading
df_1 = df_tick.iloc[:5]
df_2 = df_tick.iloc[5:] # tested 20-50 (did not work). works until 46, 49-50 works fine 9-50 are missing now
df_3 = df_tick.iloc[51:100]
df_3 = df_tick.iloc[101:200]
df_4 = df_tick.iloc[201:]
print("Shape of new dataframes - {} , {} , {} , {}".format(df_1.shape, df_2.shape, df_3.shape, df_4.shape))

In [ ]:
#Example tickers: ticker_list = ['BZ6.F',	'DUE.DE',	'EXL.DE',	'COP.DE',	'NFN.DE',	'MED.SW',
#                'SAP.DE',	'EVT.DE',	'AAG.DE',	'LPK.DE',	'OSP2.DE',	'ADN1.DE',	'SHF.DE',	
#               'SANT.DE',	'AOF.DE'] # example list
# Dict. ticker_info_dict has to be created to store information coming from .info function
# Within dict. values have to be iterable (lists, tuples, dict., sets) values returned from .info have to be 
# transformed into list. List of 308 ticker took circa 25 min.

def info(df):
    ticker_info_dict = {}
    ticker_info_dict2 = {}
    info_list = []
    total_number = 0
    total_passed = 0
    
    for tick in df:
        try:
            ticker_item = yf.Ticker(tick)
            ticker_info_dict = ticker_item.info
            print('Starting to loop through dict. of', tick)

            for key, value in ticker_info_dict.items():
                ticker_info_dict2[key] = [value]

            a = pd.DataFrame.from_dict(ticker_info_dict2)
            info_list.append(a)
               
            total_number = total_number + 1
            print('Nr: ' + str(total_number), tick)
        except Exception as e:
            total_passed = total_passed + 1
            print('Nr: ' + str(total_passed), tick, 'Passed!', e)
            #passed_df.append(tick, e)
            #passed_df.append([ticker, e]) # e text of exception, to understand reason why passed
            pass
    df_info = pd.concat(info_list, sort=False)  # transform info_list into df
    df_info.fillna(0)
    
    
    print('Start saving stock data...')

    #Create variables for specific naming of file, to not overwrite data
    #Later file shall be overwritten or appended for purpose of updating
    now = datetime.datetime.now()
    now = now.strftime("%Y.%m.%d %H_%M_%S")
    len_df = str(len(df))

    df_info.to_csv('df_info_'+len_df+'_'+now+'.csv')

    print('Stock data saved!')

In [ ]:
def fin(df):
        balance_list = []
        cashflow_list = []
        financials_list = []
        total_number = 0
        total_passed = 0
        passed_df = []
        
        for tick in df:
            try:
                ticker = yf.Ticker(tick)
                a = ticker.balancesheet.T   # T stands for transpose
                if a.index.dtype.kind == 'M':  # Check if ticker is valid, if no data can be found then index.dtype
                                               # is an object and index is filled 
                                               # with ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
                    a.insert(0, 'ticker', tick)
                    b = ticker.cashflow.T
                    b.insert(0, 'ticker', tick)
                    c = ticker.financials.T
                    c.insert(0, 'ticker', tick)

                    balance_list.append(a)
                    cashflow_list.append(b)
                    financials_list.append(c)

                    total_number = total_number + 1
                    print('Nr: ' + str(total_number), tick)
                else:
                    total_passed = total_passed + 1
                    print('Nr: ' + str(total_passed), tick, 
                          'Passed! Ticker is empty or has no datetime index with index.dtype:', a.index.dtype)
            except Exception as e:
                total_passed = total_passed + 1
                print('Nr: ' + str(total_passed), tick, 'Passed!', e)
                #passed_df.append(tick, e)
                #passed_df.append([ticker, e]) # e text of exception, to understand reason why passed
                pass

        df_balance = pd.concat(balance_list, sort=False)
        df_balance.insert(1, 'Date', pd.DatetimeIndex(df_balance.index).date) # Add try except for date column issue
        df_balance.fillna(0)

        df_cashflow = pd.concat(cashflow_list, sort=False)
        df_cashflow.insert(1, 'Date', pd.DatetimeIndex(df_cashflow.index).date)
        df_cashflow.fillna(0)

        df_financials = pd.concat(financials_list, sort=False)
        df_financials.insert(1, 'Date', pd.DatetimeIndex(df_financials.index).date)
        df_financials.fillna(0)
        try:
            df_passed = pd.concat(passed_df, sort=False)
        except Exception as e:
            print(e)
            pass
        print('Start saving stock data...')
        #df_pass = pd.DataFrame(Screener.globalpass, index=None, columns={'Ticker', 'Error'})
        
        #Create variables for specific naming of the files, to not overwrite data
        #Later files shall be overwritten or appended for purpose of updating
        now = datetime.datetime.now()
        now = now.strftime("%Y.%m.%d %H_%M_%S")
        len_df = str(len(df))
        
        df_balance.to_csv('balance_'+len_df+'_'+now+'.csv')
        df_cashflow.to_csv('cashflow_'+len_df+'_'+now+'.csv')
        df_financials.to_csv('financials_'+len_df+'_'+now+'.csv')
        
        print('Stock data saved!')
        
        #Writer not needed, as analysis will be conducted with CSV format
        #writer = pd.ExcelWriter('merge.xlsx')
        #df_balance.to_excel(writer, 'Balances', index=False)
        #df_cashflow.to_excel(writer, 'Cashflows', index=False)
        #df_financials.to_excel(writer, 'Financials', index=False)
        #writer.save()

In [ ]:
df_tick = xlsx_ticker('SnP500_open.xlsx', 'ticker')
df_tick

In [ ]:
info(df_tick)

In [ ]:
fin(df_tick)

In [ ]:
def info_essentials(df)
    ticker_info_dict = {}
    ticker_info_dict2 = {}
    info_list = []
    total_number = 0
    total_passed = 0
    
    for tick in df:
        try:
            ticker_item = yf.Ticker(tick)
            ticker_info_dict = ticker_item.info # define here the few needed items to speed up the download
            print('Starting to loop through dict. of', tick)

            for key, value in ticker_info_dict.items():
                ticker_info_dict2[key] = [value]

            a = pd.DataFrame.from_dict(ticker_info_dict2)
            info_list.append(a)
               
            total_number = total_number + 1
            print('Nr: ' + str(total_number), tick)
        
        except Exception as e:
            total_passed = total_passed + 1
            print('Nr: ' + str(total_passed), tick, 'Passed!', e)
            #passed_df.append(tick, e)
            #passed_df.append([ticker, e]) # e text of exception, to understand reason why passed
            pass
        
    df_info = pd.concat(info_list, sort=False)  # transform info_list into df
    df_info.fillna(0)
    df_info.to_excel('test.info.xlsx')

In [ ]:
# Within dict. values have to be iterable (lists, tuples, dict., sets)
#df = pd.DataFrame(data=dic_msft)
dic2 = {}
for key, value in dic_msft.items():
    dic_msft[key] = [value]

In [ ]:
df_msft = pd.DataFrame.from_dict(dic_msft)

In [ ]:
df_msft

In [ ]:
df_msft.to_excel("msft.info.xlsx")

In [ ]:
# show income statement
dic_msft_fin={}
#dic_msft_fin = msft.financials
#msft.quarterly_financials

# show balance heet
dic_msft_bal={}
dic_msft_bal=msft.balance_sheet
msft.quarterly_balance_sheet

# show cashflow
dic_msft_cash={}
#dic_msft_cash=msft.cashflow
#msft.quarterly_cashflow

# show earnings
msft.earnings
msft.quarterly_earnings

# show sustainability
msft_profile=msft.sustainability

# show analysts recommendations
msft.recommendations

# show major holders
msft.major_holders

# show institutional holders
msft.institutional_holders

In [ ]:
# df['symbol']
# user input = 'AAPL'
# df.loc[df['symbol']==user input]
# if len(df.loc[df['symbol']==user input]) ==1
#    print('The following company was found:')
#    elif len(df.loc[df['symbol']==user input]) > 1
# df = df.loc
    

In [ ]:
# Selection idea
def user_selection():
    for index, item in enumerate(options):
                print(f"Option {index + 1} - {item}")
            while True:
                try: # Is used to avoid errors
                    choice = int(input("Selection: "))
                    if choice >= 1 and choice <= len(options):
                        return options[choice-1]
                except:
                    continue